In [4]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "cifar-10-python/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [5]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [36]:
# x1 y1 得到 hidden1_1
x1 = tf.placeholder(tf.float32, [None, 3072])
y1 = tf.placeholder(tf.int64, [None])
hidden1_1 = tf.layers.dense(x1, 100, activation=tf.nn.relu)

# x2 y2 得到 hidden1_2
x2 = tf.placeholder(tf.float32, [None, 3072])
y2 = tf.placeholder(tf.int64, [None])
hidden1_2 = tf.layers.dense(x2, 100, activation=tf.nn.relu)

# hidden1_1 和 hidden1_2 拼接通过DNN得到20*10的输出 取均值后得到w 20*1
hidden_concat = tf.concat([hidden1_1, hidden1_2], 1)
hidden_concat_1 = tf.layers.dense(hidden_concat, 100, activation=tf.nn.relu)
hidden_concat_2 = tf.layers.dense(hidden_concat_1, 100, activation=tf.nn.relu)
hidden_concat_3 = tf.layers.dense(hidden_concat_2, 50, activation=tf.nn.relu)
hidden_res = tf.layers.dense(hidden_concat_3, 10)
w = tf.reduce_mean(hidden_res, axis=1)
w = tf.reshape(w, (20, 1))
# hidden1_1 = w * hidden1_1 + hidden1_1
# 结果为20*100
hidden1_1 = w * hidden1_1 + hidden1_1
hidden1_2 = w * hidden1_2 + hidden1_2

# hidden1_1 hidden1_2 继续通过DNN......待定
hidden2_1 = tf.layers.dense(hidden1_1, 100, activation=tf.nn.relu)
hidden3_1 = tf.layers.dense(hidden2_1, 50, activation=tf.nn.relu)
y_1 = tf.layers.dense(hidden3_1, 10)

hidden2_2 = tf.layers.dense(hidden1_2, 100, activation=tf.nn.relu)
hidden3_2 = tf.layers.dense(hidden2_2, 50, activation=tf.nn.relu)
y_2 = tf.layers.dense(hidden3_2, 10)

# 计算损失
loss_1 = tf.losses.sparse_softmax_cross_entropy(labels=y1, logits=y_1)
loss_2 = tf.losses.sparse_softmax_cross_entropy(labels=y2, logits=y_2)

# 计算准确率
accuracy_1 = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_1, 1), y1), tf.float64))
accuracy_2 = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_2, 1), y2), tf.float64))

# 选择合适的损失函数
loss = loss_1 * 0.9 + loss_2 * 0.1
accuracy = accuracy_1 * 0.9 + accuracy_2 * 0.1

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [37]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100

# train: 100k: 51.%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, loss_val_1, loss_val_2, acc_val, acc_val_1, acc_val_2,  _ = sess.run(
            [loss, loss_1, loss_2, accuracy, accuracy_1, accuracy_2, train_op],
            feed_dict={
                x1: batch_data,
                y1: batch_labels,
                x2: batch_data,
                y2: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, loss_1: %4.5f, loss_2: %4.5f, acc: %4.5f, acc_1: %4.5f, acc_2: %4.5f' 
                  % (i+1, loss_val, loss_val_1, loss_val_2, acc_val, acc_val_1, acc_val_2))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x1: test_batch_data, 
                        y1: test_batch_labels,
                        x2: test_batch_data, 
                        y2: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))

[Train] Step: 500, loss: 1.52339, loss_1: 1.52182, loss_2: 1.53748, acc: 0.46500, acc_1: 0.45000, acc_2: 0.60000
[Train] Step: 1000, loss: 1.89492, loss_1: 1.91282, loss_2: 1.73384, acc: 0.35000, acc_1: 0.35000, acc_2: 0.35000
[Train] Step: 1500, loss: 1.95454, loss_1: 1.94273, loss_2: 2.06085, acc: 0.34000, acc_1: 0.35000, acc_2: 0.25000
[Train] Step: 2000, loss: 1.81560, loss_1: 1.81284, loss_2: 1.84045, acc: 0.35500, acc_1: 0.35000, acc_2: 0.40000
[Train] Step: 2500, loss: 2.24051, loss_1: 2.25260, loss_2: 2.13177, acc: 0.08000, acc_1: 0.05000, acc_2: 0.35000
[Train] Step: 3000, loss: 1.53105, loss_1: 1.52736, loss_2: 1.56425, acc: 0.35500, acc_1: 0.35000, acc_2: 0.40000
[Train] Step: 3500, loss: 1.28734, loss_1: 1.29242, loss_2: 1.24163, acc: 0.55500, acc_1: 0.55000, acc_2: 0.60000


KeyboardInterrupt: 